<a href="https://colab.research.google.com/github/WangKun00293/bart-bert-project/blob/main/BERT_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
ag_news = load_dataset("ag_news")
## load dataset
import pandas as pd
df_train=ag_news["train"].to_pandas()
test=ag_news["test"].to_pandas()
from sklearn.model_selection import train_test_split
sample = df_train.sample(n=10000, random_state=1)
train, val = train_test_split(sample, test_size=0.1, random_state=311)
train_0=train[train["label"]==0].sample(n=500, random_state=323)
train_1=train[train["label"]==1].sample(n=800, random_state=323)
train_2=train[train["label"]==2].sample(n=1200, random_state=323)
train_3=train[train["label"]==3].sample(n=1500, random_state=323)
df_train=pd.concat([train_0,train_1,train_2,train_3]).sample(frac=1, random_state=324).reset_index(drop=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data preparation

In [ ]:
import re
def clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'[&\\@#\$%\^*()_+={}\[\]:;"\'<>\|`~]', '', text)

    return text

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
# don't need prompt for BERT, add it for fine-tuning BART
label_map = {0: "Write a World News", 1: "Write a Sport News", 2: "Write a Business News", 3: "Write a Science/Technology News"}
df_train['prompt'] = df_train['label'].map(label_map)
df_remain, df_BART= train_test_split(df_train, test_size=0.5, random_state=325)
df_test, df_val = train_test_split(test,test_size=0.5,random_state=325)
df_remain=df_remain.reset_index(drop=True)
df_BART=df_BART.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

## Base-line model with BERT

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
MAX_LENGTH = 200
BATCH_SIZE = 16
train_dataset = TextDataset(dataframe=df_train, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_dataset = TextDataset(dataframe=df_val, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataset = TextDataset(dataframe=df_test, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
learning_rate = 2e-05
optimizer = torch.optim.AdamW(params=bert_model.parameters(), lr=learning_rate)
epochs = 8
bert_model.train()
import os

# Directory where you want to save your models
model_save_path = "/content/drive/MyDrive/planB/bert"
os.makedirs(model_save_path, exist_ok=True)

for epoch in range(epochs):
    bert_model.train()
    total_loss = 0
    # Training loop
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # Validation loop
    bert_model.eval()  # Switch to evaluation mode
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

    # Save model after each epoch
    epoch_save_path = os.path.join(model_save_path, f'BERT402_epoch_{epoch+1}')
    os.makedirs(epoch_save_path, exist_ok=True)
    bert_model.save_pretrained(epoch_save_path)
    bert_tokenizer.save_pretrained(epoch_save_path)


Epoch: 1, Training Loss: 0.4685771725475788, Validation Loss: 0.336482368941818
Epoch: 2, Training Loss: 0.2060338284522295, Validation Loss: 0.3236728911483739
Epoch: 3, Training Loss: 0.11114253040030599, Validation Loss: 0.38833237435872187
Epoch: 4, Training Loss: 0.07684591680392623, Validation Loss: 0.3953743933958291
Epoch: 5, Training Loss: 0.03921186407934874, Validation Loss: 0.42878560513304936
Epoch: 6, Training Loss: 0.0382522312887013, Validation Loss: 0.48368759450557475
Epoch: 7, Training Loss: 0.024055518636479974, Validation Loss: 0.4949544692150818
Epoch: 8, Training Loss: 0.015631473707966505, Validation Loss: 0.5323910228418922


# load model and evaluate

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

def evaluate(model, dataloader):
    model.eval()  # Put the model in evaluation mode

    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_accuracy = accuracy_score(true_labels, predictions)
    print(f'Validation Accuracy: {avg_accuracy}')

    # Detailed classification report
    print("\nClassification Report:\n", classification_report(true_labels, predictions, target_names=['Class1', 'Class2', 'Class3', 'Class4']))

    # Confusion Matrix
    print("Confusion Matrix:\n", confusion_matrix(true_labels, predictions))

In [ ]:
evaluate(bert_model, test_loader)

Validation Accuracy: 0.9086842105263158

Classification Report:
               precision    recall  f1-score   support

      Class1       0.94      0.88      0.91       935
      Class2       0.97      0.98      0.97       928
      Class3       0.90      0.85      0.87       967
      Class4       0.84      0.93      0.88       970

    accuracy                           0.91      3800
   macro avg       0.91      0.91      0.91      3800
weighted avg       0.91      0.91      0.91      3800

Confusion Matrix:
 [[821  26  35  53]
 [  5 911   5   7]
 [ 34   0 819 114]
 [  9   4  55 902]]
